In [1]:
#Importing needed libraries
import requests
import pandas as pd
from datetime import datetime
import geopandas as gpd
import fiona
import time
from json import JSONDecodeError
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from statsmodels.discrete.count_model import ZeroInflatedPoisson
from personal_lib import general_functions as gf


from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="capstone_test")

C:\Users\johnf\AppData\Roaming\Python\Python310\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
## There are mulitple sources of yearly data. Compiling the source URLs here as well as the API urls to grab
## privately owned buildings over 25,000 ft2 and in City-owned buildings over 10,000 ft2
building_energy_LL84_sources={"2022+":{
    "api":"https://data.cityofnewyork.us/resource/5zyy-y8am.json",
    "info":"https://data.cityofnewyork.us/Environment/NYC-Building-Energy-and-Water-Data-Disclosure-for-/5zyy-y8am/about_data"
                              },
                              "2021":{
    "api":"https://data.cityofnewyork.us/resource/7x5e-2fxh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/7x5e-2fxh/about_data"
                              },
                              "2020":{
    "api":"https://data.cityofnewyork.us/resource/usc3-8zwd.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/usc3-8zwd/about_data"
                                  },
                              "2019":{
    "api":"https://data.cityofnewyork.us/resource/wcm8-aq5w.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/wcm8-aq5w/about_data"
                                  },
                              "2018":{
    "api":"https://data.cityofnewyork.us/resource/4tys-3tzj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4tys-3tzj/about_data"
                                   },
                              "2017":{
    "api":"https://data.cityofnewyork.us/resource/4t62-jm4m.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/4t62-jm4m/about_data"
                                  },
                              "2016":{
    "api":"https://data.cityofnewyork.us/resource/utpj-74fz.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/utpj-74fz/about_data"
                                  },
                              "2015":{
    "api":"https://data.cityofnewyork.us/resource/77q4-nkfh.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/77q4-nkfh/about_data"
                                  },
                              "2014":{
    "api":"https://data.cityofnewyork.us/resource/nbun-wekj.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/nbun-wekj/about_data"
                                  },
                              "2013":{
    "api":"https://data.cityofnewyork.us/resource/yr5p-wjer.json",
    "info":"http://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/yr5p-wjer"
                                  },
                              "2012":{
    "api":"https://data.cityofnewyork.us/resource/r6ub-zhff.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/r6ub-zhff/about_data"
                                  },
                              "2011":{
    "api":"https://data.cityofnewyork.us/resource/k7nh-aufb.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/k7nh-aufb/about_data"
                                  },
                              "2010":{
    "api":"https://data.cityofnewyork.us/resource/kswi-37bp.json",
    "info":"https://data.cityofnewyork.us/Environment/Energy-and-Water-Data-Disclosure-for-Local-Law-84-/kswi-37bp/about_data"
                                  }
}


In [8]:
PAGE = 1000               # <-- enforce 1,000 rows per page
TIMEOUT = 30
MAX_RETRIES = 5
BACKOFF_BASE = 1.5

session = requests.Session()
headers = {}

def fetch_all_rows_1k(api_url: str, source_years: str, source_info_url: str) -> pd.DataFrame:
    offset = 0
    frames = []

    while True:
        params = {"$limit": PAGE, "$offset": offset}

        for attempt in range(1, MAX_RETRIES + 1):
            try:
                resp = session.get(api_url, params=params, headers=headers, timeout=TIMEOUT)
                if resp.status_code in (429, 502, 503, 504):
                    time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))
                    continue
                resp.raise_for_status()

                if "json" not in resp.headers.get("Content-Type", "").lower():
                    preview = resp.text[:200]
                    raise ValueError(f"Non-JSON response (status {resp.status_code}): {preview}")

                data_chunk = resp.json()
                if not data_chunk:
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

                # Create DataFrame
                df = pd.DataFrame(data_chunk)

                # 🔑 Add your metadata columns here
                df["source_years"] = source_years
                df["source_api_url"] = api_url
                df["source_info_url"] = source_info_url

                frames.append(df)

                # If less than PAGE, stop; otherwise keep paginating
                if len(data_chunk) < PAGE:
                    return pd.concat(frames, ignore_index=True)

                offset += PAGE
                break  # Success, go to next page

            except (requests.RequestException, JSONDecodeError, ValueError) as e:
                if attempt == MAX_RETRIES:
                    print(f"⚠️ Failed fetching {api_url} at offset {offset}: {e}")
                    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
                time.sleep(BACKOFF_BASE ** attempt * (0.1 * attempt))


In [9]:
agg_running_list = []
for k, v in building_energy_LL84_sources.items():
    print(f"Fetching {k} -> {v['api']}")
    df = fetch_all_rows_1k(v["api"], k, v["info"])
    if not df.empty:
        agg_running_list.append(df)
    else:
        print(f"Warning: no rows returned for {k} ({v['api']}).")

nyc_building_energy = pd.concat(agg_running_list, ignore_index=True) if agg_running_list else pd.DataFrame()
print(f"Total rows: {len(nyc_building_energy)}")

Fetching 2022+ -> https://data.cityofnewyork.us/resource/5zyy-y8am.json
Fetching 2021 -> https://data.cityofnewyork.us/resource/7x5e-2fxh.json
Fetching 2020 -> https://data.cityofnewyork.us/resource/usc3-8zwd.json
Fetching 2019 -> https://data.cityofnewyork.us/resource/wcm8-aq5w.json
Fetching 2018 -> https://data.cityofnewyork.us/resource/4tys-3tzj.json
Fetching 2017 -> https://data.cityofnewyork.us/resource/4t62-jm4m.json
Fetching 2016 -> https://data.cityofnewyork.us/resource/utpj-74fz.json
Fetching 2015 -> https://data.cityofnewyork.us/resource/77q4-nkfh.json
Fetching 2014 -> https://data.cityofnewyork.us/resource/nbun-wekj.json
Fetching 2013 -> https://data.cityofnewyork.us/resource/yr5p-wjer.json
Fetching 2012 -> https://data.cityofnewyork.us/resource/r6ub-zhff.json
Fetching 2011 -> https://data.cityofnewyork.us/resource/k7nh-aufb.json
Fetching 2010 -> https://data.cityofnewyork.us/resource/kswi-37bp.json
Total rows: 305906


In [12]:
nyc_building_energy["report_year"].unique()

array(['2022', '2023', nan], dtype=object)

In [18]:
nyc_building_energy["source_years"].unique()

array(['2022+', '2021', '2020', '2019', '2018', '2017', '2016', '2015',
       '2014', '2013', '2012', '2011', '2010'], dtype=object)

In [21]:
nyc_building_energy.groupby(["source_years"]).agg({"property_id":"nunique","source_api_url":"count"})

,property_id,source_api_url
source_years,,
2010,0,10329
2011,0,12400
2012,13197,13382
2013,12985,13133
2014,13024,13052
2015,15835,16139
2016,16339,16378
2017,22681,34686
2018,24820,25245


In [57]:
residential_chunk3.groupby(["source_years"]).agg({"property_id":"nunique","source_api_url":"count"})

,property_id,source_api_url
source_years,,
2010,0,10329
2011,0,12400
2012,804,804
2013,9291,9405
2014,9416,9435
2015,6834,6908
2016,7059,7071
2017,11948,18323
2018,13378,13696


In [34]:
test = nyc_building_energy[nyc_building_energy['source_years']=="2010"]
# test['primary_property_type'].unique()

In [36]:
print(test.columns)

test2 = test.dropna(how='all',axis=1)
test2.columns

Index(['report_year', 'property_id', 'property_name', 'parent_property_id',
       'parent_property_name', 'year_ending', 'nyc_borough_block_and_lot',
       'nyc_building_identification', 'address_1', 'city',
       ...
       'building_id', 'period_ending_date', 'state',
       'national_average_site_eui', 'national_average_source_eui',
       'multifamily_home_dishwashers', 'multifamily_home_market_rate',
       'hospital_number_of_licensed', 'building_profile_status',
       'total_on_site_electric'],
      dtype='object', length=843)


Index(['address_1', 'city', 'weather_normalized_site_eui',
       'fuel_oil_2_use_kbtu', 'fuel_oil_4_use_kbtu', 'diesel_2_use_kbtu',
       'district_steam_use_kbtu', 'natural_gas_use_kbtu',
       'data_center_gross_floor_area', 'data_center_it_energy',
       ...
       'building_id', 'period_ending_date', 'state',
       'national_average_site_eui', 'national_average_source_eui',
       'multifamily_home_dishwashers', 'multifamily_home_market_rate',
       'hospital_number_of_licensed', 'building_profile_status',
       'total_on_site_electric'],
      dtype='object', length=166)

In [55]:
test["parent_property_id"].unique()

array([nan], dtype=object)

In [48]:
residential_chunk3[residential_chunk3["primary_property_type_self"]=='Multifamily Housing'] ## confirming onlyl multifam. No nulls here




,report_year,property_id,property_name,parent_property_id,parent_property_name,year_ending,nyc_borough_block_and_lot,nyc_building_identification,address_1,city,...,period_ending_date,state,national_average_site_eui,national_average_source_eui,multifamily_home_dishwashers,multifamily_home_market_rate,hospital_number_of_licensed,building_profile_status,total_on_site_electric,year_ending_year
1,2022,9793770,1870 Pelham Parkway South,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31,2042500026,2047795,1870 Pelham Parkway South,Bronx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
5,2022,14377690,1680 Ocean Ave,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31,3067300001,3180535,1680 Ocean Ave,Brooklyn,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
8,2022,15176247,88-24 Merrick Blvd,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31,4098150067,4210063,88-24 Merrick Blvd,Jamaica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
9,2022,15176327,90-11 149th Street,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31,4096790052,4206819,90-11 149 str,Jamaica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
10,2022,15176328,148-25 89th Ave,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31,4096930051,4207100,148-25 89 Ave,Jamaica,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305901,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Arlington,...,2010-12-31T00:00:00.000,Virginia,44.5,148.6,NaN,NaN,NaN,NaN,NaN,NaN
305902,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,210 W 55th St,New York,...,2010-12-31T00:00:00.000,New York,128.3,264.1,NaN,NaN,NaN,NaN,NaN,NaN
305903,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,132 West 45th St,New York,...,2010-12-31T00:00:00.000,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305904,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,Arlington,...,2010-12-31T00:00:00.000,VA,44.5,148.6,NaN,NaN,NaN,NaN,NaN,NaN


In [56]:
    ## Using nulls instead of "not Available".
    chunk = nyc_building_energy.replace("Not Available",np.nan)
    # Convert year_ending to datetime
    chunk["year_ending"] = pd.to_datetime(chunk["year_ending"], errors="coerce")
    # Optional: extract just the year if that’s all you need
    chunk["year_ending_year"] = chunk["year_ending"].dt.year
    # print(chunk["year_ending_year"].unique())
    
    ### Property Types
    chunk["primary_property_type"] = chunk["primary_property_type"].combine_first(chunk["primary_property_type_epa"])
    chunk["primary_property_type_self"] = chunk["primary_property_type_self"].combine_first(chunk["primary_property_type_self_selected"])
    chunk = chunk.drop(columns=["primary_property_type_self_selected","primary_property_type_epa"])


    ## Beginning of Limiting to Multifamily Homes (Step 1)
    residential_chunk = chunk[(((chunk["list_of_all_property_use"].isin([i for i in chunk["list_of_all_property_use"].unique() if 'Multifamily Housing' in str(i)]))
            |(chunk["list_of_all_property_use"].isnull()))
           &((chunk["primary_property_type_self"]=="Multifamily Housing")|(chunk["primary_property_type_self"].isnull()))
           &((chunk["primary_property_type"]=="Multifamily Housing")|(chunk["primary_property_type"].isnull())))]
    
    ## Second Step is limiting to NON MIXED USE, so ONLY MultiFamily Residences
    residential_chunk2 = residential_chunk[((residential_chunk['list_of_all_property_use'].isnull())
                                            | residential_chunk['list_of_all_property_use'].apply(
                                                lambda x: len(x.split(",")) == 1 if isinstance(x, str) else False
                                            ))]


    ## Limtiing to the Metered Areas for whole property or whole building
    residential_chunk3 = residential_chunk2[((residential_chunk2["metered_areas_energy"].isin(['Whole Building', "Whole Property"]))
                                             |(residential_chunk2["metered_areas_energy"].isnull()))]

    residential_chunk3 = residential_chunk3[((residential_chunk3["construction_status"]=='Existing')|(residential_chunk3["construction_status"].isnull()))]
    
    ## Stand Alone Properties
    residential_chunk3 = residential_chunk3[((residential_chunk3["parent_property_id"].isnull())
                                             |(residential_chunk3["parent_property_id"]=='Not Applicable: Standalone Property'))]
    ## Dropping what i already filtered on
    residential_chunk3 = residential_chunk3.drop(columns=["primary_property_type_self","primary_property_type",
                                                          "national_median_reference", "list_of_all_property_use",
                                                          "largest_property_use_type","construction_status",
                                                          "parent_property_id","parent_property_name"])
    

C:\Users\johnf\AppData\Local\Temp\ipykernel_68152\1684718292.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  chunk = nyc_building_energy.replace("Not Available",np.nan)
C:\Users\johnf\AppData\Local\Temp\ipykernel_68152\1684718292.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  chunk["year_ending_year"] = chunk["year_ending"].dt.year


In [20]:
nyc_building_energy.head()

,report_year,property_id,property_name,parent_property_id,parent_property_name,year_ending,nyc_borough_block_and_lot,nyc_building_identification,address_1,city,...,building_id,period_ending_date,state,national_average_site_eui,national_average_source_eui,multifamily_home_dishwashers,multifamily_home_market_rate,hospital_number_of_licensed,building_profile_status,total_on_site_electric
0,2022,6414946,58-01 Grand Avenue,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31T00:00:00.000,4026780001,4059918,58-01 Grand Avenue,Queens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022,9793770,1870 Pelham Parkway South,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31T00:00:00.000,2042500026,2047795,1870 Pelham Parkway South,Bronx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022,10177418,Central Building,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31T00:00:00.000,4157040040,4300145,2011 Mott Ave,Far Rockaway,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022,13511507,215 East 99th Street,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31T00:00:00.000,1016490009,1052383,215 East 99th Street,New York,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022,14009907,23-25 31 Street,Not Applicable: Standalone Property,Not Applicable: Standalone Property,2022-12-31T00:00:00.000,4008350027,4017180,23-25 31 Street,Queens,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
[i for i in nyc_building_energy.columns if ('year' in i or 'source' in i)]

nyc_building_energy[['report_year','year_ending',"year_s_labeled",'source_years','source_api_url','source_info_url']].drop_duplicates()


,report_year,year_ending,year_s_labeled,source_years,source_api_url,source_info_url
0,2022,2022-12-31T00:00:00.000,NaN,2022+,https://data.cityofnewyork.us/resource/5zyy-y8...,https://data.cityofnewyork.us/Environment/NYC-...
30485,2023,2023-12-31T00:00:00.000,NaN,2022+,https://data.cityofnewyork.us/resource/5zyy-y8...,https://data.cityofnewyork.us/Environment/NYC-...
64169,NaN,2021-12-31T00:00:00.000,NaN,2021,https://data.cityofnewyork.us/resource/7x5e-2f...,https://data.cityofnewyork.us/Environment/Ener...
94011,NaN,2020-12-31T00:00:00.000,NaN,2020,https://data.cityofnewyork.us/resource/usc3-8z...,https://data.cityofnewyork.us/Environment/Ener...
122078,NaN,2019-12-31T00:00:00.000,NaN,2019,https://data.cityofnewyork.us/resource/wcm8-aq...,https://data.cityofnewyork.us/Environment/Ener...
...,...,...,...,...,...,...
302850,NaN,NaN,"2008, 2011",2010,https://data.cityofnewyork.us/resource/kswi-37...,https://data.cityofnewyork.us/Environment/Ener...
303999,NaN,NaN,"2001, 2002, 2003, 2005, 2006, 2008, 2010",2010,https://data.cityofnewyork.us/resource/kswi-37...,https://data.cityofnewyork.us/Environment/Ener...
304001,NaN,NaN,"2001, 2008, 2010",2010,https://data.cityofnewyork.us/resource/kswi-37...,https://data.cityofnewyork.us/Environment/Ener...
304002,NaN,NaN,"2006, 2008",2010,https://data.cityofnewyork.us/resource/kswi-37...,https://data.cityofnewyork.us/Environment/Ener...


In [59]:
nyc_building_energy.to_csv("nyc_dob_energy_2010_2024.csv",index=False)